# Práctica 1. Parte 2. _Apendicitis_

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

## 1. Enunciado
El objetivo de esta práctica es entrenar una neurona artiﬁcial para que sea capaz de detectar si un paciente puede o menos sufrir de apendicitis.

En primer lugar hay que descargar los archivos appendicitis.dat y muestra\_pacientes.dat de la base de datos keel Dataset ([enlace](https://sci2s.ugr.es/keel/dataset.php?cod=183))

In [7]:
# Carga de los datos del dataset de entrenamiento:
data = pd.read_csv('appendicitis/data.csv')

In [8]:
# Dividimos en porcentaje de entrenamiento y prueba:
train_percent = 0.8
test_percent = 1 - train_percent

train_data = data.sample(frac=train_percent)
test_data = data.copy().drop(train_data.index)

Una vez la neurona pueda leer el archivo se pide:

1\. Añadir como opción una función de salida que proceda de f (p) = sin(p) en [−1, 1] con imagen en el intervalo apropiado.

In [10]:
# Función 1
# TODO: normalizar los valores de entrada y salida para que en el intervalo [-1,1] de x, la imagen esté entre 0 y 1, es decir (f(-1)=0) y (f(1)=1)
def sin(x):
    """Calcula el seno de un número x, con imagen entre 0 y 1.
    Precondiciones: x ∈ [-1,1]"""
    # Función seno en intervalo [-1,1]
    return math.sin(x)/2 + 0.5


2\. Añadir como opción una función de salida que proceda de f(_p_) = 1/(1+ _e_<sup>−p</sup>) en (0, 1), con imagen en
el intervalo apropiado

In [ ]:
# Función 2
def fun2(x):
    """Calcula el valor de la función f(x) = 1/(1 + e^(-x)), con 
    imagen entre 0 y 1.
    Precondiciones: x ∈ (0, 1)"""
    return 1/(1 + math.exp(-x))

3\. Añadir como opción una función de salida que proceda de f (p) = ex p(−p<sup>2</sup>) en (0, 1] con imagen en el
intervalo apropiado